This code uses Principal Component Analysis (to reduce dimensionality) and k-means clustering to group similar 'hit' songs together.

In [ ]:
# Import Libraries

import pandas as pd
import requests
import numpy as np
import io
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
import seaborn as sns; sns.set()
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Downloading the csv file from our GitHub repo

url = "https://raw.githubusercontent.com/Jingxue-24/QM2-team9/main/Spotify%20API/merged.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Previewing data

print (df.head())
print(df.columns)


                                 Name  ... Time_signature
0          Bridge Over Troubled Water  ...              4
1      (They Long To Be) Close To You  ...              4
2  American Woman - 7" Single Version  ...              4
3                War - Single Version  ...              4
4       Ain't No Mountain High Enough  ...              4

[5 rows x 17 columns]
Index(['Name', 'Album', 'Artist', 'Popular_date', 'Release_date', 'Length',
       'Popularity', 'Danceability', 'Acousticness', 'Danceability.1',
       'Energy', 'Instrumentalness', 'Liveness', 'Loudness', 'Speechiness',
       'Tempo', 'Time_signature'],
      dtype='object')


All works well. Moving on to prepping the data for PCA.

In [ ]:
df.drop(['Name', 'Album', 'Artist','Popular_date','Release_date', 'Popularity', 'Danceability.1', 'Liveness'], axis=1, inplace=True)
print(df.columns)  
print(df.head()) 
df.dropna(inplace=True)

Index(['Length', 'Danceability', 'Acousticness', 'Energy', 'Instrumentalness',
       'Loudness', 'Speechiness', 'Tempo', 'Time_signature'],
      dtype='object')
   Length  Danceability  Acousticness  ...  Speechiness    Tempo  Time_signature
0  293120         0.149        0.8220  ...       0.0323   79.764               4
1  276000         0.543        0.6690  ...       0.0344   89.078               4
2  231640         0.538        0.0424  ...       0.0375   93.077               4
3  200960         0.591        0.0677  ...       0.1920  109.481               4
4  215200         0.393        0.1910  ...       0.0448   98.757               4

[5 rows x 9 columns]


All works well. Moving on to Kaiser-Meyer-Olkin test for suitability of data for PCA.

In [ ]:
# KMO test
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all, kmo_model = calculate_kmo(df)
kmo_model # = 0.5958992986526002 

ModuleNotFoundError: ignored

Not a great KMO value - "miserable". We will have to discuss this in the website. 

In [ ]:
from sklearn.preprocessing import StandardScaler
#Standardising data
std_df = StandardScaler().fit_transform(df) #normalizing

#PCA
pca = PCA(n_components=9)
principalComponents = pca.fit_transform(std_df)

#Plotting the variances for each PC
PC = range(1, pca.n_components_+1)
fig, ax = plt.subplots(figsize=(8,8)) # width & height 
plt.bar(PC, pca.explained_variance_ratio_, color='gold')
ax.set(title="How much variance is explained by PCs", xlabel="Principal Components", ylabel="Variance %")
plt.xticks(PC)

#Putting it in a dataframe
PCA_components = pd.DataFrame(principalComponents)

First principal component explains over 25% of the variance. Second explains approximately 13%.

In [ ]:
plt.scatter(PCA_components[0], PCA_components[1], alpha=.3, color='gold')
plt.xlabel=('PCA 1')
plt.ylabel=('PCA 2')
plt.show()

Now, we need to use k-means to see how many clusters there actually are. 

In [ ]:
# Clustering - seeing exactly how many there are. 
inertias = []

for k in range(1,10):
  model = KMeans(n_clusters=k)

  model.fit(PCA_components.iloc[:,:3])

  inertias.append(model.inertia_)

fig, ax = plt.subplots(figsize=(8,8)) # width & height 
plt.plot(range(1,10), inertias, '-p', color='gold')
ax.set(title="Elbow Method for Optimal Number of k", xlabel="k", ylabel="Sum of Squared Distances")
plt.show()

Elbow method suggests 3 groupings.

In [ ]:
model = KMeans(n_clusters=3)
model.fit(PCA_components.iloc[:,:2])
labels = model.predict(PCA_components.iloc[:,:2])
colormap = np.array(['r', 'g', 'b'])
fig, ax = plt.subplots(figsize=(8,8)) # width & height 
plt.scatter(PCA_components[0], PCA_components[1], c=colormap[labels])
ax.set(title="K-Means Clustering on Principal Components", 
        xlabel="Principal Component 1",
        ylabel="Principal Component 2")
plt.show()

In [ ]:
# Adding the the cluster number to the original csv file. 

kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
y_kmeans = kmeans.fit_predict(std_df)

kmeans = pd.DataFrame(data=y_kmeans, dtype=int)
kmeans.columns=['k_cluster']
print(kmeans.shape)
kmeans.head()

In [ ]:
# Downloading the original full csv file from our GitHub repo

url = "https://raw.githubusercontent.com/Jingxue-24/QM2-team9/main/merged.csv"
download = requests.get(url).content
full_df = pd.read_csv(io.StringIO(download.decode('utf-8')))
full_df.drop(['Release_date','Popularity', 'Danceability.1'], axis=1, inplace=True)
# Concatenating
df_cluster = pd.concat([full_df, kmeans], axis=1)
print(df_cluster.shape)
df_cluster.head() # it works - have the cluster with the name of the song. 

In [ ]:
# Check for null
df_cluster.isnull().sum() #no nulls!

#Number of songs per cluster
df_cluster['k_cluster'].value_counts()

# Statistical distribution of the data in each column, for each cluster 
df_cluster.groupby("k_cluster").describe()

In [ ]:
# Updated datasets
cluster_zero = df_cluster.loc[df_cluster['k_cluster']==0]
cluster_one = df_cluster.loc[df_cluster['k_cluster']==1]
cluster_two = df_cluster.loc[df_cluster['k_cluster']==2]

# Exporting
cluster_zero.to_csv('cluster_zero.csv', index=False, header=True)
cluster_one.to_csv('cluster_one.csv', index=False, header=True)
cluster_two.to_csv('cluster_two.csv', index=False, header=True)
df_cluster.to_csv('cluster_all.csv', index=False, header=True)

Now, summarising the three clusters. 

First, scatter plots between three audio features - acousticness, length and tempo. 

In [ ]:
# Reading the files
k_zero = pd.read_csv('cluster_zero.csv')
k_one = pd.read_csv('cluster_one.csv')
k_two = pd.read_csv('cluster_two.csv')
k_all = pd.read_csv('cluster_all.csv')

In [ ]:
# Scatter for k_all - acoustic and length
import plotly.express as px
fig = px.scatter(k_all, x="Acousticness", y="Length", color='k_cluster',
                 size='Tempo', hover_name="Name", hover_data=["Artist"], labels={'k_cluster':'Cluster'})
fig.show()

# Export as html for website
fig.write_html("./k_all_acoustic_length.html", include_plotlyjs='cdn') 

Comparing the spread of years of the songs in each of the three clusters 

In [ ]:
import plotly.graph_objects as go
import numpy as np

k0_cols = k_zero["Popular_date"]
k1_cols = k_one["Popular_date"]
k2_cols = k_two["Popular_date"]

fig = go.Figure()
# Using x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=k0_cols, name="Cluster Zero"))
fig.add_trace(go.Box(x=k1_cols, name="Cluster One"))
fig.add_trace(go.Box(x=k2_cols, name="Cluster Two"))
 
fig.show()

# Export as html for website
fig.write_html("./cluster_years_spread.html", include_plotlyjs='cdn') 

In [ ]:
# Scatter for k_all - dance and energy
fig = px.scatter(k_all, x="Danceability", y="Energy", color='k_cluster',
                 size='Tempo', hover_name="Name", hover_data=["Artist"], labels={'k_cluster':'Cluster'})
fig.show()

# Export as html for website
fig.write_html("./k_all_dance_energy.html", include_plotlyjs='cdn') 

Tutorial(s) used:


*   https://medium.com/@dmitriy.kavyazin/principal-component-analysis-and-k-means-clustering-to-visualize-a-high-dimensional-dataset-577b2a7a5fe2 
*   https://jakevdp.github.io/PythonDataScienceHandbook/05.11-k-means.html
*   https://plotly.com/python/ 
*   https://towardsdatascience.com/machine-learning-algorithms-part-9-k-means-example-in-python-f2ad05ed5203 



